In [45]:
import config
import dataloader

import pandas as pd

In [46]:
conf = config.Config()
print(conf.DATA_PATH)
train_df = pd.read_csv(conf.DATA_PATH+'/train.csv')

D:/Downloads/imaterialist-fashion-2020-fgvc7


In [47]:
d = dataloader.Dataset(conf, train_df)
print(d.TRAINING_DATA_PATH)

D:/Downloads/imaterialist-fashion-2020-fgvc7/train/


In [48]:
a,b,c = d.get_next_batch()

['217a3f82eddc3c2b5e8163baac78c6d5' 'ecb43b14c86bc8aa394d028c0f4a8ff4'
 '565d2031ea14228235c10de9cef8578d' 'd476303ff53356238a1452cca0ed2419'
 '7951541b77c37b5fb1b7c8b2155e1251' '12772b7b70cc37b05b2b2c67b7e96f20'
 'd1a94ba85bf1b0cc915348db656a7017' 'fc1cad584430981822507562677fabf4']


# Model

In [49]:
import torchvision
import torch.nn as nn
import copy

In [50]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
myModel = copy.deepcopy(model)

# The model has 4 main parts
<ol>
<li>transform(GeneralizedRCNNTransform)</li> 
<li>backbone(BackbonewithFPN and FPN-FeaturePyramidNetwork)</li>
<li>rpn (RegionProposalNetwork)</li>
<li>roi_heads - RoIHeads</li>
</ol>

We need to change roi_heads as it has TwoMLPHead, FastRCNNPredictor, MaskRCNNHeads, MaskRCNNPredictor ()

In [51]:
model.rpn.eval() #model.backbone.eval()

RegionProposalNetwork(
  (anchor_generator): AnchorGenerator()
  (head): RPNHead(
    (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (cls_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
    (bbox_pred): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [52]:
model.roi_heads.eval()

RoIHeads(
  (box_roi_pool): MultiScaleRoIAlign()
  (box_head): TwoMLPHead(
    (fc6): Linear(in_features=12544, out_features=1024, bias=True)
    (fc7): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (box_predictor): FastRCNNPredictor(
    (cls_score): Linear(in_features=1024, out_features=91, bias=True)
    (bbox_pred): Linear(in_features=1024, out_features=364, bias=True)
  )
  (mask_roi_pool): MultiScaleRoIAlign()
  (mask_head): MaskRCNNHeads(
    (mask_fcn1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (mask_fcn2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (mask_fcn3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (mask_fcn4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu4): ReLU(inplace=True)
  )
  (mask_predictor): MaskRCNNPredictor(
    (conv5_mask): ConvTr

In [55]:
model.roi_heads.eval()
myModel.roi_heads.box_predictor.cls_score = nn.Linear(1024, conf.NO_OF_CLASSES)
myModel.roi_heads.box_predictor.bbox_pred = nn.Linear(1024, 4*conf.NO_OF_CLASSES)
myModel.roi_heads.mask_predictor.mask_fcn_logits = nn.Conv2d(256, conf.NO_OF_CLASSES, kernel_size=(1, 1), stride=(1, 1))
myModel.roi_heads.eval()

RoIHeads(
  (box_roi_pool): MultiScaleRoIAlign()
  (box_head): TwoMLPHead(
    (fc6): Linear(in_features=12544, out_features=1024, bias=True)
    (fc7): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (box_predictor): FastRCNNPredictor(
    (cls_score): Linear(in_features=1024, out_features=46, bias=True)
    (bbox_pred): Linear(in_features=1024, out_features=184, bias=True)
  )
  (mask_roi_pool): MultiScaleRoIAlign()
  (mask_head): MaskRCNNHeads(
    (mask_fcn1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (mask_fcn2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (mask_fcn3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (mask_fcn4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu4): ReLU(inplace=True)
  )
  (mask_predictor): MaskRCNNPredictor(
    (conv5_mask): ConvTr